# GRIDFINDER WORKSHOP

This notebook will guide you through the use of the _gridfinder_ model to create predictions on the location of medium-voltage grid lines in a given country, based on night-time lights data and OpenStreetMap road networks. This version has been simplified for the workshop.

For more information, please see:
- https://github.com/carderne/gridfinder
- http://blogs.worldbank.org/energy/using-night-lights-map-electrical-grid-infrastructure
- https://engineering.fb.com/connectivity/electrical-grid-mapping/

# 1. Data Preparation
## 1.a. Import necessary Modules
If this fails, you may need to install additional modules, or check that gridfinder is in your path.

In [ ]:
# The first group is stanard Python library imports
import os
from pathlib import Path

# The second group is third-party libraries
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import display, Markdown
import numpy as np
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
import folium
import branca

# And this final group is our own code
import gridfinder as gf
from gridfinder import save_raster
print("All imports working!")

This is just a bit of code to make our raster plots nicer:

In [ ]:
def imshow(image, **kwargs):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(image, **kwargs)
    ax.axis('off')

## 1.b. Set parameters
Basic parameters used to run the model

In [ ]:
# The country name here must match the names used in the "data" folder.
country = "kenya"

## 1.c. Set folder paths

In [ ]:
# These are the input data sources
data = Path("data")
aoi_in = data / f"{country}.gpkg"
roads_in = data / "roads.gpkg"
ntl_in = data / "ntl.tif"

In [ ]:
# These are the intermediate output file paths
outputs = Path("outputs")
ntl_out = outputs / "ntl_clipped.tif"
ntl_thresh_out = outputs / "ntl_thresh.tif"
targets_out = outputs / "targets.tif"
roads_out = outputs / "roads.tif"
dist_out = outputs / "dist.tif"
guess_out = outputs / "guess.tif"

In [ ]:
# These are the final output files
guess_skeletonized_out = outputs / "guess_skel.tif"
guess_vec_out = outputs / "guess.gpkg"
animate_out = outputs / "animated"

# 2. Geoprocessing and Algorithm Tuning

## 2.a. Clip night-time lights
In this step, we clip the night-time lights.

First, we read in the AOI (area of interest) for our specified country. These are stored in individual GeoPackages (a type of vector data file) for each country. We read them in using [GeoPandas](http://geopandas.org/) (aliased to `gpd`), which is an easy Python library for dealing with vector data, built on top of the well-known Pandas library.

These files contain sub-national boundaries, so we use `dissolve` to convert it into a single boundary for the whole country.

In [ ]:
aoi = gpd.read_file(aoi_in)
aoi_diss = aoi.dissolve(by="admin")
print("Dissolved")

Then, before we start using our AOI on the raster imagery, we `buffer` it a little. This is to ensure that we don't get any "edge effects" like distortions on the boundaries.

In [ ]:
buff = aoi_diss.copy()
buff.geometry = aoi_diss.buffer(0.1)
print("Buffered")

Let's have a look at the three GeoDataFrames that we now have.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15,5))
aoi.plot(ax=ax1,)
aoi_diss.plot(ax=ax2, color="red")
buff.plot(ax=ax3, color="green")
ax1.set_title("Original")
ax2.set_title("Dissolved")
ax3.set_title("Buffered")

Finally we're ready to clip the night-time lights raster (which is provided as a single file for all of our countries. Have a look at lines 94-148 in [gridfinder/_util.py](files/gridfinder/gridfinder/_util.py) to see the code for clipping. It used GeoPandas to extract the geometry, and then [Rasterio](https://rasterio.readthedocs.io/en/stable/) to mask the night-time lights raster.

In this process, we get three results:
- A 2D numpy array containing the values
- An 'affine' transformation, those mathematically shows how these values translate to real world coordinates
- a CRS (Coordinate Reference System) defining the geographical reference being used

In [ ]:
ntl_clipped, affine, crs = gf.clip_raster(ntl_in, buff)
print("Shape:", ntl_clipped.shape)
print("Affine transformation matrix:\n", affine)
print("Coordinate reference system:", crs)

Then we simply save the raster (see the code in lines 19-54 in [gridfinder/_util.py](files/gridfinder/gridfinder/_util.py).

In [ ]:
gf.save_raster(ntl_out, ntl_clipped, affine, crs, nodata=None)
print("Raster saved!")

## 2.b. Filter night-time lights
Now we're ready to do something with our clipped night-time lights!

First, we create a filter that is used to highlight areas of the night-time lights that are significantly brighter that their surroundings. The filter is of the form:  

In [ ]:
# f = 1/(1+|d|)^3, where d != 0
# f = 0          , where d == 0

where *d* is that pixel's distance from a given square's centroid. The filter is normalised so that sum(f) == 0, and then it is subtracted from the original image. To see the code, please see lines 106-126 in [gridfinder/prepare.py](files/gridfinder/gridfinder/prepare.py). The filter created is actually a 2D array of values with 41 rows and 41 columns.

In [ ]:
ntl_filter = gf.create_filter()
print("Shape:", ntl_filter.shape)

The following code is purely to make a visualiztion of the 2D filter.

In [ ]:
X = np.fromfunction(lambda i, j: i, ntl_filter.shape)
Y = np.fromfunction(lambda i, j: j, ntl_filter.shape)
fig = plt.figure()
sns.set()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, ntl_filter, cmap=cm.coolwarm, linewidth=0, antialiased=False)

Then, we apply that filter to every location in our clipped night-time lights raster. This function `prepare_ntl` can be found in lines 129-212 in [gridfinder/prepare.py](files/gridfinder/gridfinder/prepare.py). It does two main things:
- Apply the filter to the raster
- Resample the raster to a higher or lower resolution

In this workshop we want things to go quickly, so we keep a lower resolution. But for better results, we could operate at a much higher resolution! (Note: higher resolution doesn't automatically mean better, and we can't _add_ any detail to the raster, but some of the later steps can definitely benefit from a higher resolution.)

But first we need to set two important parameters! Firstly, `ntl_threshold`: this is applied _after_ filtering, and values above this are considered electrified. If you change this value, the results will probably not be great, but feel free to experiment!

In [ ]:
ntl_threshold = 0.1

Then there is the `scale_factor`, which controls the resolution. A positive value creates a high-resolution, and vice-versa. If the model takes too long to run, you could try lowering this.

In [ ]:
scale_factor = 1

And finally, we're ready to prepare the night-time lights.

In [ ]:
ntl_thresh, affine = gf.prepare_ntl(ntl_out,
                                    buff,
                                    ntl_filter=ntl_filter,
                                    threshold=ntl_threshold,
                                    upsample_by=scale_factor)

Remember when we buffered our AOI earlier? Now that we're done with the risk of edge-effects, it's time to clip our raster to the original `aoi_diss` polygon for our country.

In [ ]:
save_raster(ntl_thresh_out, ntl_thresh, affine)
targets, affine, _ = gf.clip_raster(ntl_thresh_out, aoi_diss)
gf.save_raster(targets_out, targets, affine)
print("Targets prepared")

Then we show what our results look like. In theory, each white pixel is a location with electricity access!

In [ ]:
imshow(targets, cmap="gray")

## 2.c. Roads: assign values, clip and rasterize
We take the raw roads data from OSM and assign different 'costs' to different classes of roads. This means the algorithm will prefer to follow larger roads (motorways) over smaller roads or empty land.

The code for `prepare_roads` is in lines 308-374 of [gridfinder/prepare.py)[files/gridfinder/gridfinder/prepare.py).

We have to supply `targets_out` (our electrification targets from above) to the function, so that it knows to make sure that the roads_raster is _exactly_ the same shape as our targets.

In [ ]:
roads_raster, affine = gf.prepare_roads(roads_in,
                                        aoi_in,
                                        targets_out)

In [ ]:
print("Targets raster shape:", targets.shape)
print("Costs raster shape:", roads_raster.shape)

As before, we save our work. The image below should show a faint outline of the roads values we've just created. The darker a pixel is, the cheaper it is to cross it.

In [ ]:
save_raster(roads_out, roads_raster, affine, nodata=-1)
print("Costs prepared")
imshow(roads_raster, cmap='Reds_r', vmin=0, vmax=1)

## 2.d. Get targets and costs and run algorithm

This function simply loads the rasters that we've created, and gets the `start` point: this is just the first point (closest to top-left) that contains an electrified location. The gridfinder algorithm will start there.

In [ ]:
targets, costs, start, affine = gf.get_targets_costs(targets_out, roads_out)

Finally! This is the most interesting part of the whole process. Here we use our modified Dijkstra minimum-spanning tree to start at `start` and recursively search for points to electrify.

The code is in lines 91-229 of [gridfinder/gridfinder.py](files/gridfinder/gridfinder/gridfinder.py).

This might take a little while. If it takes too long, go up and change the `scale_by` factor to something less than 1 (try 0.5 or even less).

In [ ]:
dist = gf.optimise(targets, costs, start,
                   jupyter=True,
                   animate=True,
                   affine=affine,
                   animate_path=animate_out)
print("Done!")

And again, we save and display the results. The darker blues areas are the areas more likely to be grid locations.

In [ ]:
save_raster(dist_out, dist, affine)
imshow(dist, cmap="Blues_r", vmin=0, vmax=50)

# 3. Visualizing Results
## 3.a. Filter dist results to grid guess
Then from this result, we extract the locations below the given cutoff. These are assumed to be locations that have MV infrastructure.

First, we choose a cutoff value. Values _below_ this will be considered to be actual medium-voltage grid lines. By default it is 0.0. You can try a higher value, but you might lose a lot of grid! You can see the code for this in lines 30-66 of [gridfinder/post.py](files/gridfinder/gridfinder/post.py).

In [ ]:
cutoff = 0.0

In [ ]:
guess, affine = gf.threshold(dist_out, cutoff=cutoff)
save_raster(guess_out, guess, affine)
print("Cutoff applied!")

The result from this still has lots of big blobs, but we only want lines. So we _skeletonize_ it: that is, we thin out the blobs so only lines remain. This in lines 69-103 of [gridfinder/post.py](files/gridfinder/gridfinder/post.py) and is basically just a wrapper of a [skimage.morphology.skeletonize](https://scikit-image.org/docs/dev/api/skimage.morphology#skimage.morphology.skeletonize).

In [ ]:
guess_skel, affine = gf.thin(guess_out)
save_raster(guess_skeletonized_out, guess_skel, affine)

Once again, ket's see what it looks like.

In [ ]:
print("Got guess and skeletonized")
imshow(guess_skel, cmap="gray_r")

## 3.c. Convert to geometry
To make it easier to work with, we convert this raster result into a vector. A raster after all is just a 2D array of values, which doesn't make sense or a bunch of _lines_. So we convert it into a vector line geometry.

First of all we use `raster_to_lines`, which comes from lines 106-174 of [gridfinder/post.py](files/gridfinder/gridfinder/post.py). This is a pretty complicated function, but basically it loops through the raster, looking for grid cells that are connected and converting them into geometry.

In [ ]:
mv = gf.raster_to_lines(guess_skeletonized_out)
mv.crs = {"init": "epsg:4326"}
print("Converted to geometry")

As always, we save and preview.

In [ ]:
mv.to_file(guess_vec_out, driver="GPKG")
fig, ax = plt.subplots(figsize=(6,6))
mv.plot(ax=ax)
ax.axis('off')
plt.show()

That's very pretty, but it would be much nicer to see what it looks like overlaid on a real map. For that, we use a really great library called [folium](https://python-visualization.github.io/folium/) to create a JavaScript map without having to leave our notebook.

Note you can zoom in, pan around and see if the result actually makes sense.

In [ ]:
minx, miny, maxx, maxy = list(mv.bounds.iloc[0])
bounds = ((miny, minx), (maxy, maxx))

m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
folium.GeoJson(mv).add_to(m)
m

# 4. Applying zonal statitics
Now it would be nice to learn a bit about the distribution of our distribution lines. We use something called 'zonal statistics': basically, statistics from our raster (like mean, sum, max, median...) but applied for separate zones. In this case, our zones are going to be the sub-national regions from our AOI.

First we use the function `zonal_stats` from the library [rasterstats](https://github.com/perrygeo/python-rasterstats) to calculate these statistics. Note that the `vectors` parameter contains our zones, while the `raster` parameter contains the data to apply the statistics to.

In [ ]:
stats = zonal_stats(
    vectors=aoi,
    raster=guess_skeletonized_out,
    affine=affine,
    stats='count'
)
print("Calculated zonal stats")

Now let's insert these into our AOI GeoDataFrame from before.

In [ ]:
aoi["count_mv"] = [int(s["count"]) for s in stats]
print("Let's see the results.")
aoi[["admin", "name", "count_mv"]].head(2)

That's fine, but the results simply show the number of MV cells in each zone. We want to know the kilometres of MV-lines! To do that, there a few steps. First we need to know that our affine transformation (from earlier) contains the pixel width as its first element. Voila:

In [ ]:
width_deg = affine[0]
print(width_deg)

However, this is in degrees, not kilometres! The real solution would take into account the fact that degrees are not the same size everywhere, but let's take an easier way. At the equator, 1 degree is roughly equal to 100km. Therefore:

In [ ]:
width = width_deg * 100
print(width)

Now we can multiple this value across our GeoDataFrame to get the length of MV lines in each cell

In [ ]:
aoi["mv_length_km"] = aoi["count_mv"] * width
aoi[["admin", "name", "count_mv", "mv_length_km"]].head(4)

Now let's put it on a map so we can see how it looks.

In [ ]:
colorscale = branca.colormap.linear.YlOrRd_09.scale(0, aoi["mv_length_km"].max())
colorscale.caption = 'Total MV per region'

In [ ]:
m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
def style_function(feature):
    sum_mv = feature["properties"]["mv_length_km"]
    return {
        'fillOpacity': 0.5,
        'weight': 1,
        'color': "black",
        'fillColor': colorscale(sum_mv)
    }
folium.GeoJson(
    aoi,
    style_function = style_function
).add_to(m)

m.add_child(colorscale)
m

# 5. Proximity to grid
Finally for something even more interesting, let's use some population data to see what percentage of the population lives within a certain distance of the predicted grid.

Let's load our population data (which is for the whole of Africa) and see the total population in our selected country.

In [ ]:
# Load population data
pop_in = data / "ghs_africa.tif"
pop_total = zonal_stats(vectors=aoi_diss, raster=pop_in, stats='sum')[0]["sum"]
print("Total population:", pop_total)

Then we buffer the predicted MV grid by a set amount. First choose an amount

In [ ]:
buff_dist = 1  # in km

Now, remember that our vector is in degrees. To accurately buffer it by a certain number of kilometres, we should re-project it into a different CRS that uses metres or kilometres, do the buffer, and then re-project it back... To make it easier, we're going to do what we did before and remember that (at the equator), 1 degree is approximately 100 km.

In [ ]:
buff_dist_deg = buff_dist * 0.01  # approximate

Then we're ready to bufer the geometry by the specified amount.

In [ ]:
mv_buff = mv.copy()
mv_buff.geometry = mv_buff.buffer(buff_dist_deg)

And again we use `zonal_stats` to calculate. Except this time, notice that the `vector` is now our buffered MV lines (these are the zones we're interested in), while the `raster` is the population data (the data about which we want statistics.)

In [ ]:
# Calculate zonal population statistics
pop_stats = zonal_stats(
    vectors=mv_buff,
    raster=pop_in,
    stats='sum'
)
pop_mv = pop_stats[0]["sum"]
print("Calculated stats.")

And let's see what it looks like!

In [ ]:
print(f"Total population:\t{pop_total:,.0f}")
print(f"Population within {buff_dist} km:\t{pop_mv:,.0f}")
print(f"Percentage within {buff_dist} km:\t{100*pop_mv/pop_total:.0f}%")

# 6. Finished!
Feel free to play around with different parameters, or dig into the gridfinder code and change anything you want. There's a very good chance you'll find bugs or have ideas for improvement.

If so, you're very welcome to submit issues and pull requests at the official gridfinder repo: https://github.com/carderne/gridfinder

Many thanks from the team:
- Christopher Arderne
- Kadeem Khan
- Damon Civin